# Lane Detection using Vision Transformer

## Abstract

In this notebook, we perform Lane Detection by fine tuning a Vision Transformer Model on the OpenLaneV2 dataset. This task is essential for many downstream tasks like Bird's Eye View Representation, Motion Planning, 3D Occupancy Detection etc. 

## Dataset Description

The OpenLane dataset consists of multi-view images taken by a cameras mounted on an ego vehicle. The dataset provides annotations for Lane lines, Traffic elements,Area elements(Pedestrian Crossing, Sidewalk etc)

Dataset Link - [here](https://github.com/OpenDriveLab/OpenLane-V2/blob/master/data/README.md#download)

### RTDETR for Object Detection

In [2]:
import torch
import requests
from PIL import Image
from transformers import RTDetrForObjectDetection, RTDetrImageProcessor

In [3]:
url = 'http://images.cocodataset.org/val2017/000000039769.jpg' 
image = Image.open(requests.get(url, stream=True).raw)

image_processor = RTDetrImageProcessor.from_pretrained("PekingU/rtdetr_r50vd")
model = RTDetrForObjectDetection.from_pretrained("PekingU/rtdetr_r50vd")

inputs = image_processor(images=image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

results = image_processor.post_process_object_detection(outputs, target_sizes=torch.tensor([image.size[::-1]]), threshold=0.3)

for result in results:
    for score, label_id, box in zip(result["scores"], result["labels"], result["boxes"]):
        score, label = score.item(), label_id.item()
        box = [round(i, 2) for i in box.tolist()]
        print(f"{model.config.id2label[label]}: {score:.2f} {box}")


preprocessor_config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/172M [00:00<?, ?B/s]

sofa: 0.97 [0.14, 0.38, 640.13, 476.21]
cat: 0.96 [343.38, 24.28, 640.14, 371.5]
cat: 0.96 [13.23, 54.18, 318.98, 472.22]
remote: 0.95 [40.11, 73.44, 175.96, 118.48]
remote: 0.92 [333.73, 76.58, 369.97, 186.99]


### ResNet for Laneline Detection

In [47]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset
import os
import sys

In [61]:
import pandas as pd

data_dict_subset_A = pd.read_pickle('../../OpenLane-V2/data/OpenLane-V2/data_dict_subset_A.pkl')

In [85]:
data_dict_train = {}
data_dict_val = {}

for i in data_dict_subset_A.keys():
    if i[0] == 'train':
        data_dict_train[i] = data_dict_subset_A[i]
    if i[0] == 'val':
        data_dict_val[i] = data_dict_subset_A[i]
    

In [42]:
# Resnet 50
from torchvision.models import resnet50, ResNet50_Weights

model = resnet50(weights=ResNet50_Weights.DEFAULT)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /Users/akshaybharadwaj/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:29<00:00, 3.45MB/s]


### Project 3d to 2d 

In [36]:
# ==============================================================================
# Binaries and/or source for the following packages or projects 
# are presented under one or more of the following open source licenses:
# utils.py    The OpenLane-V2 Dataset Authors    Apache License, Version 2.0
#
# Contact wanghuijie@pjlab.org.cn if you have any issue.
#
# Copyright (c) 2023 The OpenLane-V2 Dataset Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

import numpy as np


THICKNESS = 4

COLOR_DEFAULT = (0, 0, 255)
COLOR_DICT = {
    0:  COLOR_DEFAULT,
    1:  (255, 0, 0),
    2:  (0, 255, 0),
    3:  (255, 255, 0),
    4:  (255, 0, 255),
    5:  (0, 128, 128),
    6:  (0, 128, 0),
    7:  (128, 0, 0),
    8:  (128, 0, 128),
    9:  (128, 128, 0),
    10: (0, 0, 128),
    11: (64, 64, 64),
    12: (192, 192, 192),
}


def interp_arc(points, t=1000):
    r'''
    Linearly interpolate equally-spaced points along a polyline, either in 2d or 3d.

    Parameters
    ----------
    points : List
        List of shape (N,2) or (N,3), representing 2d or 3d-coordinates.
    t : array_like
        Number of points that will be uniformly interpolated and returned.

    Returns
    -------
    array_like  
        Numpy array of shape (N,2) or (N,3)

    Notes
    -----
    Adapted from https://github.com/johnwlambert/argoverse2-api/blob/main/src/av2/geometry/interpolate.py#L120

    '''
    
    # filter consecutive points with same coordinate
    temp = []
    for point in points:
        point = point.tolist()
        if temp == [] or point != temp[-1]:
            temp.append(point)
    if len(temp) <= 1:
        return None
    points = np.array(temp, dtype=points.dtype)

    assert points.ndim == 2

    # the number of points on the curve itself
    n, _ = points.shape

    # equally spaced in arclength -- the number of points that will be uniformly interpolated
    eq_spaced_points = np.linspace(0, 1, t)

    # Compute the chordal arclength of each segment.
    # Compute differences between each x coord, to get the dx's
    # Do the same to get dy's. Then the hypotenuse length is computed as a norm.
    chordlen = np.linalg.norm(np.diff(points, axis=0), axis=1)  # type: ignore
    # Normalize the arclengths to a unit total
    chordlen = chordlen / np.sum(chordlen)
    # cumulative arclength

    cumarc = np.zeros(len(chordlen) + 1)
    cumarc[1:] = np.cumsum(chordlen)

    # which interval did each point fall in, in terms of eq_spaced_points? (bin index)
    tbins = np.digitize(eq_spaced_points, bins=cumarc).astype(int)  # type: ignore

    # #catch any problems at the ends
    tbins[np.where((tbins <= 0) | (eq_spaced_points <= 0))] = 1  # type: ignore
    tbins[np.where((tbins >= n) | (eq_spaced_points >= 1))] = n - 1

    s = np.divide((eq_spaced_points - cumarc[tbins - 1]), chordlen[tbins - 1])
    anchors = points[tbins - 1, :]
    # broadcast to scale each row of `points` by a different row of s
    offsets = (points[tbins, :] - points[tbins - 1, :]) * s.reshape(-1, 1)
    points_interp = anchors + offsets
#     print(f"points interp : {points_interp}")
    return points_interp



In [77]:
def _project(points, intrinsic, extrinsic):
    if points is None:
        return points
    
    points_in_cam_cor = np.linalg.pinv(np.array(extrinsic['rotation'])) \
        @ (points.T - np.array(extrinsic['translation']).reshape(3, -1))
    
#     print("points cam 1 : ", points_in_cam_cor)
    
    points_in_cam_cor = points_in_cam_cor[:, points_in_cam_cor[2, :] > 0]
#     print("points cam : ", points_in_cam_cor)
    
    if points_in_cam_cor.shape[1] > 1:
        points_on_image_cor = np.array(intrinsic['K']) @ points_in_cam_cor
        points_on_image_cor = points_on_image_cor / (points_on_image_cor[-1, :].reshape(1, -1))
        points_on_image_cor = points_on_image_cor[:2, :].T
    else:
        points_on_image_cor = None
#     print(" points_on_image_cor : ", points_on_image_cor)
    return points_on_image_cor

### Custom Dataset

In [82]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import os

class Lane3DDataset(Dataset):
    def __init__(self, image_dir, data_dict, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.annotations = self.load_annotations(data_dict)

    def load_annotations(self, data_dict):
        annotations = []
        try:
            for ann in data_dict:
    #             print(data_dict[ann].keys())
                frame = data_dict[ann]['sensor']['ring_front_center']
                image_name = frame['image_path']
                intrinsics = frame['intrinsic']
                extrinsics = frame['extrinsic']
                lane_points_3d = data_dict[ann]['annotation']['lane_segment'][0]['centerline']
                lane_points_2d = _project(interp_arc(np.array(lane_points_3d)), frame['intrinsic'],frame['extrinsic'])
                if type(center_points) == np.ndarray:
                    annotations.append((image_name, intrinsics, extrinsics, lane_points_2d))
            print(f"len of annotations : {len(annotations)}")
            return annotations
        except:
            print(data_dict[ann].keys())

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        image_name, intrinsics, extrinsics, lane_points_2d = self.annotations[idx]
        img_path = os.path.join(self.image_dir, image_name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        lane_points_2d = torch.tensor(lane_points_2d, dtype=torch.float32)

        return image, intrinsics, extrinsics, lane_points_2d


In [87]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = Lane3DDataset(image_dir='../../OpenLane-V2/data/OpenLane-V2/mapless_driving/', data_dict=data_dict_train, transform=transform)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)


len of annotations : 3199


In [88]:
dataset.annotations

[('train/00000/image/ring_front_center/315967376899927209.jpg',
  {'K': array([[1.77733411e+03, 0.00000000e+00, 7.78222656e+02],
          [0.00000000e+00, 1.77733411e+03, 1.01621350e+03],
          [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
   'distortion': array([-0.24358925, -0.20579766,  0.32205908])},
  {'rotation': array([[-9.57540534e-04,  5.28159325e-03,  9.99985594e-01],
          [-9.99999076e-01, -9.69974438e-04, -9.52430359e-04],
          [ 9.64930115e-04, -9.99985582e-01,  5.28251716e-03]]),
   'translation': array([1.63685016, 0.00213953, 1.40590799])},
  None),
 ('train/00000/image/ring_front_center/315967377349927211.jpg',
  {'K': array([[1.77733411e+03, 0.00000000e+00, 7.78222656e+02],
          [0.00000000e+00, 1.77733411e+03, 1.01621350e+03],
          [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
   'distortion': array([-0.24358925, -0.20579766,  0.32205908])},
  {'rotation': array([[-9.57540534e-04,  5.28159325e-03,  9.99985594e-01],
          [-9.9

### 3D Lane Detection Model

In [89]:
class ResNet3DLaneDetection(nn.Module):
    def __init__(self, num_points):
        super(ResNet3DLaneDetection, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        
        # Remove the fully connected layer
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-2])
        
        # Add a global average pooling layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Add a fully connected layer to predict 3D lane points
        self.fc = nn.Linear(2048, num_points * 3)  # num_points * 3 for (x, y, z) coordinates

    def forward(self, x):
        x = self.resnet(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x.view(x.size(0), -1, 3)  # Reshape to (batch_size, num_points, 3)

# Number of points to detect per lane
num_points = 20
model = ResNet3DLaneDetection(num_points=num_points)


/Users/akshaybharadwaj/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/akshaybharadwaj/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [90]:
criterion = nn.MSELoss()  # Suitable for regression
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


### Model Training

In [91]:
import torch.optim as optim

# Model
num_points = 20
model = ResNet3DLaneDetection(num_points=num_points)
model = model.cuda()

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, intrinsics, extrinsics, lane_points_2d in dataloader:
        images = images.cuda()
        lane_points_2d = lane_points_2d.cuda()

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, lane_points_2d)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

# Save the trained model
torch.save(model.state_dict(), 'lane_detection_model.pth')


AssertionError: Torch not compiled with CUDA enabled